In [130]:
using Pkg, CSV, DataFrames, PrettyTables, CategoricalArrays

In [136]:
using StatsBase, StatsFuns, Statistics, GLM, CategoricalArrays

## This notebook
This notebook is written in Julia 1.9.1 (latest version) and presents the results of the survey posted earlier about the preferences and choices of how members of the IMIA Telehealth working group define telehealth. We informally surveyed 100+ members of the discussion group, and 19 members responded. The data were collected using a Qualtrics survery over three weeks. The annotated results and data are presented here. You can play with the data and the codes presented here if you fork this project. All codes are annotated. 

In [190]:
# read data from the teledef.csv data set stored in this repository
df = CSV.read("teledef.csv", DataFrame)
# Print the variable names
foreach(x -> println(x), names(df))

StartDate
EndDate
Status
Progress
Duration (in seconds)
Finished
RecordedDate
ResponseId
DistributionChannel
UserLanguage
Preference
Ranking_1
Ranking_2
Ranking_3
Ranking_4
Ranking_5
Other
experience
Role
Q8
Q7


In [191]:
nrow(df) # number of rows in the data set

23

In [192]:
# Select the list of variables to work with

selected_df = select(df, [:ResponseId, :Preference, :Ranking_1, :Ranking_2,
        :Ranking_3, :Ranking_4, :Ranking_5, :Other, :experience, :Q8, :Role])
nrow(selected_df) # check that the rows match

23

In [194]:
## Preferences of the definitions

ourprefs = select(selected_df, :Preference)
ourprefs2 = dropmissing(ourprefs)
ourprefs3 = ourprefs2[3:length(ourprefs2.Preference), :]
ourprefs4 = categorical(ourprefs3.Preference)
ourprefs5 = countmap(ourprefs4)
mycats = DataFrame(:category =>[k for (k,v) in ourprefs5], :counts => [v for (k,v) in ourprefs5])
mycats1 = sort(mycats, :counts, rev = true)
mycats1.pct = (mycats1.counts ./ sum(mycats1.counts) ) * 100
mycats1

Row,category,counts,pct
,Cat…,Int64,Float64
1,Distance Bridging System,5,27.7778
2,None of the above / Other,5,27.7778
3,"Distance Bridging System,Store and Forward,Mobile Health,Mobile Messaging",3,16.6667
4,"Distance Bridging System,Store and Forward,Mobile Health",2,11.1111
5,"Distance Bridging System,Mobile Health",1,5.55556
6,"Distance Bridging System,Mobile Messaging",1,5.55556
7,Store and Forward,1,5.55556


In [195]:
# define a function to create similar counts with other variables
function give_counts(x)
    x1 = countmap(x)
    x2 = DataFrame(:category => [k for (k,v) in x1], :counts => [v for (k,v) in x1])
    x3 = sort(x2, :counts, rev = true)
    x3.pct = (x3.counts  ./ sum(x3.counts) ) * 100
    return(x3)
end

give_counts (generic function with 1 method)

In [196]:
# count and percentages of how members defined telehealth for themselves

owndefs = select(selected_df, :Other)
owndefs2 = dropmissing(owndefs)
owndefs3 = owndefs2[3:length(owndefs2.Other), :]
owndefs4 = categorical(owndefs3.Other)
owndef_counts = give_counts(owndefs4)

Row,category,counts,pct
,Cat…,Int64,Float64
1,Nil,2,14.2857
2,"Provision of healthcare services, including education and administration, at a distance using ICT.",1,7.14286
3,"Delivery of services using any type of information and communication technogy, not necessary electronic or digital and where distance is more than 0",1,7.14286
4,"the options Distance Bridging System, Store and Forward, and Mobile Health can be considered examples of telehealth and/or telemedicine, while Mobile Messaging may or may not be considered a form of telehealth depending on the specific context. It can be defined as the use of telecommunication and information technologies to provide remote clinical healthcare services, medical education, and health administration. It encompasses a range of healthcare services and technologies that include virtual consultations, remote monitoring of patients, digital imaging and health record management, and mobile health applications.",1,7.14286
5,The delivery of health services where at least two entities are separated by distance,1,7.14286
6,"Telehealth is a much brother term than the definitions included. I think you are talking about telemedicine. Both terms are defined by WHO, PAHO, ATA, etc:",1,7.14286
7,"The difference should be between “health” and “medicine”. Health includes healthy people and patients, while medicine covers only sick people (patients). “Tele” is a technology term that can precede many medical disciplines.",1,7.14286
8,A telehealth service is a:\nhealthcare activity supported at a distance by information and communication technology service(s)\n \nwhere a service is service is the:\noutput of an organization with at least one activity necessarily performed between the organization and the customer\n,1,7.14286
9,"Telehealth can be described as the delivery of healthcare at a distance using telecommunication technologies. Telehealth contains both preventive and curative aspects of healthcare delivery. Telemedicine, on the other hand, is the curative or the clinical part of Telehealth and can be described as 'the process of exchanging medical information from one site to another via electronic communications to improve a patient's clinical health status. (http://doi.org/10.4038/sljbmi.v11i1.8090)",1,7.14286


In [198]:
# members' years of experience
exper = select(selected_df, :experience)
exper2 = dropmissing(exper)
exper3 = exper2[3:length(exper2.experience), :]
exper4 = categorical(exper3.experience)
exper_counts = give_counts(exper4)

Row,category,counts,pct
,Cat…,Int64,Float64
1,20,3,21.4286
2,40y,1,7.14286
3,25,1,7.14286
4,11,1,7.14286
5,15,1,7.14286
6,27,1,7.14286
7,Above 10 years,1,7.14286
8,14 years,1,7.14286
9,14,1,7.14286


In [199]:
# Obtain the summary estimates of their years of experience
years_of_work = [20, 20, 20, 40, 25, 11, 15, 27, 10, 14, 15, 20, 30]
mean_y = mean(years_of_work) # 20.5
min_y = minimum(years_of_work) #10
max_y = maximum(years_of_work) #40
med_y = median(years_of_work) # 20

work_sum = [mean_y, min_y, max_y, med_y]
work_sum

4-element Vector{Float64}:
 20.53846153846154
 10.0
 40.0
 20.0

In [200]:
# Countries and regions they belong to
country = select(selected_df, :Q8)
country2 = dropmissing(country)
country3 = country2[3:length(country2.Q8), :]
country4 = categorical(country3.Q8)
country_counts = give_counts(country4)

# country

Row,category,counts,pct
,Cat…,Int64,Float64
1,"Brisbane, Australia",2,20.0
2,Colombia,1,10.0
3,Asia-Pacific,1,10.0
4,"North-Western Province, Sri Lanka",1,10.0
5,Colombo,1,10.0
6,"Sankara Nethralaya eye hospital Chennai, India",1,10.0
7,"Manila, Philippines, University of the Philippines Manila",1,10.0
8,"LSHTM (UK), Keele university (UK), Primary Care Network (UK)",1,10.0
9,Australia,1,10.0


In [163]:
# Job role
role = select(selected_df, :Role)
role2 = dropmissing(role)
role3 = role2[3:length(role2.Role), :]
role4 = categorical(role3.Role)
role4_counts = give_counts(role4)

Row,category,counts,pct
,Cat…,Int64,Float64
1,Former director of the University of the Philippines Manila National Telehealth Center,1,7.14286
2,Specialized medical Doctor in Health Informatics with special interest in Telehealth. Development of Telemedicine Guidelines for Sri Lanka. ISO committee member in ISO/TC215 Taskforce on Telehealth and Virtual Care.,1,7.14286
3,Researcher,1,7.14286
4,"Research, evaluation and adoption.",1,7.14286
5,Implementer,1,7.14286
6,Using and teaching,1,7.14286
7,Research & Strategy,1,7.14286
8,"(Remote) Care provider, Telehealth consultant, software and systems developer, trainer and academician",1,7.14286
9,Consultant,1,7.14286


In [202]:
# Preferences with multiple entries split, define a function
function split_uniformly(v)
    s = split.(v, ',')
    n = maximum(length.(s))
    [NamedTuple(Symbol.("choice", 1:n) .=> get.(Ref(genres), 1:n, missing))
     for genres in s]
end

split_uniformly (generic function with 1 method)

In [203]:
# Split the multiple choices

choices = select(selected_df, :Preference)
choices2 = choices[3:length(choices.Preference), :]
string.(choices2.Preference)
#choices3 = select(choices2, :Preference => split_uniformly => AsTable)
typeof(choices2.Preference)
choices2.Preference
choices3 = dropmissing!(df2)
choices4 = choices3[3:length(choices3.Preference), :]
choices4
choices5 = select(choices4, :Preference => split_uniformly => AsTable)

Row,choice1,choice2,choice3,choice4
,SubStrin…,SubStrin…?,SubStrin…?,SubStrin…?
1,Distance Bridging System,Store and Forward,Mobile Health,Mobile Messaging
2,None of the above / Other,missing,missing,missing
3,Distance Bridging System,Mobile Messaging,missing,missing
4,Distance Bridging System,missing,missing,missing
5,None of the above / Other,missing,missing,missing
6,Distance Bridging System,missing,missing,missing


In [172]:
# relative values of the rankings
ranking_data = selected_df[4:23, [:Ranking_1, :Ranking_2, :Ranking_3, :Ranking_4, :Ranking_5]]
rank_data = rename(ranking_data,
    :Ranking_1 => :distance_bridging,
    :Ranking_2 => :store_forward,
    :Ranking_3 => :mobile_applications,
    :Ranking_4 => :mobile_messaging,
    :Ranking_5 => :none_above
    )

Row,distance_bridging,store_forward,mobile_applications,mobile_messaging,none_above
,String?,String?,String?,String?,String?
1,1,4,2,3,missing
2,1,2,3,4,missing
3,1,2,3,4,missing
4,1,2,3,4,missing
5,missing,missing,missing,missing,missing
6,missing,missing,missing,missing,missing
7,1,3,2,4,5
8,missing,missing,missing,missing,missing
9,1,3,2,4,5


In [183]:
# Calculate the average values for the ranks
distance_bridge_rank_av = mean(parse.(Float64, skipmissing(rank_data.distance_bridging))) # 1.13
store_fwd_rank_av = mean(parse.(Float64, skipmissing(rank_data.store_forward))) # 2.73
mob_app_rank_av = mean(parse.(Float64, skipmissing(rank_data.mobile_applications))) #2.86
mob_msg_rank_av = mean(parse.(Float64, skipmissing(rank_data.mobile_messaging))) # 3.53
none_rank_av = mean(parse.(Float64, skipmissing(rank_data.none_above))) #4.63

relative_ranks = DataFrame(:Definitions => ["distance bridge", "Store Forward", "Mobile Apps", "Mobile Messaging", "None"],
    :Rank_values => [distance_bridge_rank_av, store_fwd_rank_av, mob_app_rank_av, mob_msg_rank_av, none_rank_av] )


Row,Definitions,Rank_values
,String,Float64
1,distance bridge,1.13333
2,Store Forward,2.73333
3,Mobile Apps,2.86667
4,Mobile Messaging,3.53333
5,None,4.63636


## Summary 
Most members where 20 years or more in the profession and are engaged in a range of roles in teaching, research, and practice of telehealth. The members who responded were from different continents and different areas of expertise. There was agreement telehealth as "bridging the distance" and to some extent was about store and forward technology and mobile apps. That telehealth was also about mobile messaging was relegated to low ranks of agreement. Members also offered their own definitions of the field and there was wide range of definitions. This preliminary data exploration needs a validation with wider range of telehealth practitioners. If seasoned, self-selected practitioners of telehealth and telemedicine cannot arrive at a consensus as to the nature of their own discipline beyond the obvious (telehealth as bridging the distance), there is a need to study the viewpoint of a wider audience and this is important in terms of the practice of this discipline in the field. 